In [1]:
import json
import pandas as pd
import ijson
from datetime import datetime
import time

In [2]:
webattack_brute_start = 1499343600

In [3]:
webattack_brute_end = 1499346060

In [4]:
webattack_xss_start = 1499346900

In [5]:
webattack_xss_end = 1499348160

In [6]:
webattack_sql_start = 1499348400

In [7]:
webattack_sql_end = 1499348580

In [10]:
start_time = time.time()

dict_epochtime = {}
dict_source_ip = {}
dict_dst_ip = {}
dict_source_port = {}
dict_dst_port = {}
dict_stream = {}
dict_sequence = {}
dict_eth_header = {}
dict_ip_header = {}
dict_tcp_header = {}
dict_payload = {}
dict_direction = {}
dict_b_labels ={}
dict_d_labels = {}


total_no = 0
packets = 0
with open("Thursday.json", "rb") as f:
    for record in ijson.items(f, "item"):
        total_no += 1
        if 'eth:ethertype:ip:tcp' in record["_source"]["layers"]["frame"]["frame.protocols"]:
            packets += 1
            
            if (packets % 1000000) == 0:
                print((packets/12943381)*100,(time.time() - start_time))
                
            epoch_time = record["_source"]['layers']['frame']['frame.time_epoch']
            source_ip = record["_source"]["layers"]['ip']['ip.src']
            dst_ip = record["_source"]["layers"]['ip']['ip.dst']
            eth_raw = record["_source"]['layers']['eth_raw'][0]
            ip_raw = record["_source"]['layers']['ip_raw'][0]
            tcp_raw = record["_source"]['layers']['tcp_raw'][0]
            all_hex_raw = record["_source"]["layers"]["frame_raw"][0]
            
            all_header_raw = eth_raw + ip_raw + tcp_raw
            
            len_header = len(all_header_raw)
            
            payload_raw = all_hex_raw[len_header:]

            dict_epochtime[packets] = epoch_time
            dict_source_ip[packets] = source_ip
            dict_dst_ip[packets] = dst_ip
            dict_source_port[packets] = record["_source"]['layers']['tcp']['tcp.srcport']
            dict_dst_port[packets] = record["_source"]['layers']['tcp']['tcp.dstport']
            dict_stream[packets] = record["_source"]['layers']['tcp']['tcp.stream']
            dict_sequence[packets] = record["_source"]['layers']['tcp']['tcp.seq']
            dict_eth_header[packets] = eth_raw
            dict_ip_header[packets] = ip_raw
            dict_tcp_header[packets] = tcp_raw
            dict_payload[packets] = payload_raw



            if  float(epoch_time) >= webattack_brute_start and float(epoch_time) <= webattack_brute_end:
                ##webattack-brute = 8
                if source_ip == '172.16.0.1' and dst_ip == '192.168.10.50':

                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 8
                    dict_b_labels[packets] = 1
                elif source_ip =='192.168.10.50' and dst_ip == '172.16.0.1':
                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 8
                    dict_b_labels[packets] = 1

            elif  float(epoch_time) >= webattack_xss_start and float(epoch_time) <= webattack_xss_end:
                ##webattack-xss = 9
                if source_ip == '172.16.0.1' and dst_ip == '192.168.10.50':
                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 9
                    dict_b_labels[packets] = 1 
                elif source_ip == '192.168.10.50' and dst_ip == '172.16.0.1':
                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 9
                    dict_b_labels[packets] = 1 

            elif float(epoch_time) >= webattack_sql_start and float(epoch_time) <= webattack_sql_end:
                ##webattack_sql =10
                if source_ip == '172.16.0.1' and dst_ip == '192.168.10.50':
                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 10
                    dict_b_labels[packets] = 1 
                elif source_ip == '192.168.10.50' and dst_ip == '172.16.0.1':

                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 10
                    dict_b_labels[packets] = 1 

            #infiltration = 11
            elif source_ip == '192.168.10.8' and dst_ip == '205.174.165.73':
                dict_direction[packets]= 'fwd'
                dict_d_labels[packets] = 11
                dict_b_labels[packets] = 1 
            elif source_ip == '205.174.165.73' and dst_ip == '192.168.10.8':
                dict_direction[packets]= 'bwd'
                dict_d_labels[packets] = 11
                dict_b_labels[packets] = 1 

            
            else:
                dict_direction[packets]= 'unknown'
                dict_d_labels[packets] = 0
                dict_b_labels[packets] = 0

                
print(time.time() - start_time)

7.7259566105641175 220.86252188682556
15.451913221128235 416.04026889801025
23.17786983169235 585.3335721492767
30.90382644225647 754.6190311908722
38.629783052820585 922.6306359767914
46.3557396633847 1099.1261599063873
54.08169627394882 1272.0716280937195
61.80765288451294 1439.1168267726898
1535.9713053703308


In [17]:
pd.DataFrame(dict_d_labels.values()).value_counts()

0     7222359
11      59754
8       30059
9        9638
10        126
dtype: int64

In [18]:
packets

8538148

In [19]:
total_no

8538185

In [20]:
payload_byte_data = pd.DataFrame()

In [21]:
epoch_times_ = pd.DataFrame(dict_epochtime.values())

In [22]:
source_ips_ = pd.DataFrame(dict_source_ip.values())

In [23]:
dst_ips_ = pd.DataFrame(dict_dst_ip.values())

In [24]:
source_ports_ = pd.DataFrame(dict_source_port.values())

In [25]:
dst_ports_ = pd.DataFrame(dict_dst_port.values())

In [26]:
streams_ = pd.DataFrame(dict_stream.values())

In [27]:
seq_ = pd.DataFrame(dict_sequence.values())

In [28]:
eth_ = pd.DataFrame(dict_eth_header.values())

In [29]:
ip_ = pd.DataFrame(dict_ip_header.values())

In [30]:
tcp_ = pd.DataFrame(dict_tcp_header.values())

In [31]:
payload_ = pd.DataFrame(dict_payload.values())

In [32]:
direction_ = pd.DataFrame(dict_direction.values())

In [33]:
d_labels_ = pd.DataFrame(dict_d_labels.values())
b_labels_ = pd.DataFrame(dict_b_labels.values())

In [34]:
payload_byte_data['Epoch Time'] = epoch_times_

In [35]:
payload_byte_data['Source IP'] = source_ips_

In [36]:
payload_byte_data['Dst IP'] = dst_ips_

In [37]:
payload_byte_data['Source Port'] = source_ports_

In [38]:
payload_byte_data['Dst Port'] = dst_ports_

In [39]:
payload_byte_data['Stream Index'] = streams_

In [40]:
payload_byte_data['Sequence Number'] = seq_

In [41]:
payload_byte_data['Eth Header'] = eth_

In [42]:
payload_byte_data['IP Header'] = ip_

In [43]:
payload_byte_data['TCP Header'] = tcp_

In [44]:
payload_byte_data['Payload'] = payload_

In [45]:
payload_byte_data['Direction'] = direction_

In [46]:
payload_byte_data['Binary Label'] = b_labels_

In [47]:
payload_byte_data['Class Label'] = d_labels_

In [48]:
payload_byte_data = payload_byte_data.dropna()

In [49]:
payload_byte_data = payload_byte_data.reset_index(drop = True)

In [50]:
payload_byte_data = payload_byte_data.astype({"Source Port": int, "Dst Port": int})

In [51]:
payload_byte_data.tail()

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
7321931,1499365265.817911000,206.126.112.142,192.168.10.16,443,42198,99781,4909563,0023ae9b8abf00c1b114eb310800,450005dcf0ae0000390681a8ce7e708ec0a80a10,01bba4d6f40edf7af598c863801000b7b3290000010108...,59210587b21ed3a8d3c3c35f16fa39c781507dfc743cfc...,unknown,0.0,0.0
7321932,1499365265.818102000,206.126.112.142,192.168.10.16,443,42198,99781,4911011,0023ae9b8abf00c1b114eb310800,450005dcf0af0000390681a7ce7e708ec0a80a10,01bba4d6f40ee522f598c863801000b7168a0000010108...,c92242f4b3a7d9a4c9cc0a701d995a7ac4c682158dadd2...,unknown,0.0,0.0
7321933,1499365265.818295000,206.126.112.142,192.168.10.16,443,42198,99781,4912459,0023ae9b8abf00c1b114eb310800,45000b84f0b0000039067bfece7e708ec0a80a10,01bba4d6f40eeacaf598c863801000b7153c0000010108...,988822bedfd4cf1c3958c978619bf4fa79fcdd10d45749...,unknown,0.0,0.0
7321934,1499365265.818487000,206.126.112.142,192.168.10.16,443,42198,99781,4915355,0023ae9b8abf00c1b114eb310800,45000b84f0b2000039067bfcce7e708ec0a80a10,01bba4d6f40ef61af598c863801000b7153c0000010108...,7c8dcb764403b28792b82bce3b6e9fb9a86771a3093069...,unknown,0.0,0.0
7321935,1499365265.818679000,206.126.112.142,192.168.10.16,443,42198,99781,4918251,0023ae9b8abf00c1b114eb310800,45000b84f0b4000039067bface7e708ec0a80a10,01bba4d6f40f016af598c863801000b7153c0000010108...,cabe1f26a52e6f7dafe3dc0711c14e8b92012d526ab4f6...,unknown,0.0,0.0


In [52]:
len(payload_byte_data)

7321936

In [53]:
payload_byte_data['Direction'].value_counts()

unknown    7222359
fwd          56064
bwd          43513
Name: Direction, dtype: int64

In [54]:
payload_byte_data.groupby('Direction')['Class Label'].value_counts()

Direction  Class Label
bwd        11.0             29873
           8.0              10304
           9.0               3277
           10.0                59
fwd        11.0             29881
           8.0              19755
           9.0               6361
           10.0                67
unknown    0.0            7222359
Name: Class Label, dtype: int64

In [55]:
df_b = payload_byte_data[payload_byte_data['Class Label']==0]

In [56]:
df_a = payload_byte_data[payload_byte_data['Class Label'] != 0]

In [57]:
df_b = df_b.reset_index(drop = True)

In [58]:
df_a = df_a.reset_index(drop = True)

In [59]:
df1 = pd.read_csv('troubleshooting dataset/Thursday-WorkingHours.csv')

In [60]:
def df_prep_frwrd_prts(df1):
    
    df1 = df1[df1['Protocol'] ==6]
    df1 = df1[df1['Label'] =='BENIGN']
    #df1 = df1[['Src IP', 'Dst IP', 'Src Port', 'Dst Port', 'Timestamp','Label']].copy()
    
    #unique src prt and dst prt
    srcprt_list = df1['Src Port'].unique()
    dstprt_list = df1['Dst Port'].unique()
    print(len(srcprt_list))
    print('-----------')
    print(len(dstprt_list))
    print('-----------')
    intersect = set(srcprt_list).intersection(dstprt_list)
    print(len(intersect))
    print('-----------')
    #print(intersect)
    
    
    df1 = df1[~df1['Src Port'].isin(intersect)]
    
    #calculate the epoch time from timestamp
    #df1['epoch']=df1['Timestamp'].apply(lambda x: (datetime.strptime(x, '%d/%m/%Y %H:%M')) if x.count(':') == 1 else  (datetime.strptime(x, '%d/%m/%Y %H:%M:%S')))
    #df1['epoch']=df1['epoch'].apply(lambda x: int((datetime(x.year,x.month,x.day,(x.hour+12),x.minute,x.second)).timestamp()) if (x.hour>=1)&(x.hour<=7) else int((datetime(x.year,x.month,x.day,x.hour,x.minute,x.second)).timestamp()))
    
    df1 = df1.reset_index(drop=True)
    
    return df1

In [61]:
df1 = df_prep_frwrd_prts(df1)

36561
-----------
1640
-----------
1317
-----------


In [62]:
df_b_fwd = df_b[df_b['Source Port'].isin(df1['Src Port'].unique())] 

In [63]:
df_b_bwd = df_b[df_b['Dst Port'].isin(df1['Src Port'].unique())] 

In [64]:
df_b_fwd['Direction'] = df_b_fwd['Direction'].replace(['unknown'], 'fwd')

C:\Users\Jalal\AppData\Local\Temp\ipykernel_12768\3752041767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b_fwd['Direction'] = df_b_fwd['Direction'].replace(['unknown'], 'fwd')


In [65]:
df_b_bwd['Direction'] = df_b_bwd['Direction'].replace(['unknown'], 'bwd')

C:\Users\Jalal\AppData\Local\Temp\ipykernel_12768\3888835064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b_bwd['Direction'] = df_b_bwd['Direction'].replace(['unknown'], 'bwd')


In [66]:
df_b_bwd

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
2,1499342340.490456000,192.168.10.3,192.168.10.50,389,33898,0,1,0019b90a69f11866da9be37d0800,450001703efa400080062508c0a80a03c0a80a32,0185846a48f86a5e6b268b9f8018081d32070000010108...,000001386082013406092a864886f71201020202011100...,bwd,0.0,0.0
3,1499342340.490460000,192.168.10.3,192.168.10.50,389,33898,0,1,0019b90a69f11866da9be37d0800,450001703efa400080062508c0a80a03c0a80a32,0185846a48f86a5e6b268b9f8018081d32070000010108...,000001386082013406092a864886f71201020202011100...,bwd,0.0,0.0
8,1499342340.491375000,192.168.10.3,192.168.10.50,389,33904,1,1,0019b90a69f11866da9be37d0800,450001703efb400080062507c0a80a03c0a80a32,0185847054c096a6540cfc558018081bb47f0000010108...,000001386082013406092a864886f71201020202011100...,bwd,0.0,0.0
9,1499342340.491379000,192.168.10.3,192.168.10.50,389,33904,1,1,0019b90a69f11866da9be37d0800,450001703efb400080062507c0a80a03c0a80a32,0185847054c096a6540cfc558018081bb47f0000010108...,000001386082013406092a864886f71201020202011100...,bwd,0.0,0.0
14,1499342356.973670000,192.168.10.3,192.168.10.50,389,33898,0,317,0019b90a69f11866da9be37d0800,450001703efc400080062506c0a80a03c0a80a32,0185846a48f86b9a6b268d328018081c23c90000010108...,000001386082013406092a864886f71201020202011100...,bwd,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222354,1499365265.817911000,206.126.112.142,192.168.10.16,443,42198,99781,4909563,0023ae9b8abf00c1b114eb310800,450005dcf0ae0000390681a8ce7e708ec0a80a10,01bba4d6f40edf7af598c863801000b7b3290000010108...,59210587b21ed3a8d3c3c35f16fa39c781507dfc743cfc...,bwd,0.0,0.0
7222355,1499365265.818102000,206.126.112.142,192.168.10.16,443,42198,99781,4911011,0023ae9b8abf00c1b114eb310800,450005dcf0af0000390681a7ce7e708ec0a80a10,01bba4d6f40ee522f598c863801000b7168a0000010108...,c92242f4b3a7d9a4c9cc0a701d995a7ac4c682158dadd2...,bwd,0.0,0.0
7222356,1499365265.818295000,206.126.112.142,192.168.10.16,443,42198,99781,4912459,0023ae9b8abf00c1b114eb310800,45000b84f0b0000039067bfece7e708ec0a80a10,01bba4d6f40eeacaf598c863801000b7153c0000010108...,988822bedfd4cf1c3958c978619bf4fa79fcdd10d45749...,bwd,0.0,0.0
7222357,1499365265.818487000,206.126.112.142,192.168.10.16,443,42198,99781,4915355,0023ae9b8abf00c1b114eb310800,45000b84f0b2000039067bfcce7e708ec0a80a10,01bba4d6f40ef61af598c863801000b7153c0000010108...,7c8dcb764403b28792b82bce3b6e9fb9a86771a3093069...,bwd,0.0,0.0


In [67]:
df_all = pd.concat([df_a, df_b_fwd , df_b_bwd])

In [68]:
df_all = df_all.reset_index(drop = True)

In [69]:
df_all

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499343600.000989000,192.168.10.16,209.58.128.160,39846,443,3529,897,00c1b114eb310023ae9b8abf0800,4500003445b240004006d87ec0a80a10d13a80a0,9ba601bb201715c34e47065b8010015cc3a60000010108...,,fwd,1.0,8.0
1,1499343600.000991000,192.168.10.16,23.60.139.27,46500,80,3537,432,00c1b114eb310023ae9b8abf0800,45000034574e400040067666c0a80a10173c8b1b,b5a40050562f11268d8528b680100112f4ee0000010108...,,bwd,1.0,8.0
2,1499343600.001028000,192.168.10.16,23.60.139.27,46498,80,3536,432,00c1b114eb310023ae9b8abf0800,45000034cee640004006fecdc0a80a10173c8b1b,b5a200500271988f98a44e71801001008b460000010108...,,fwd,1.0,8.0
3,1499343600.001547000,216.58.219.226,192.168.10.25,443,49221,3561,3500,002500a8c46000c1b114eb310800,450005b92a8100003506d6dfd83adbe2c0a80a19,01bbc045d127a8cca9a506308018015d92590000010108...,170301058065e6491e2fc3dc9d004ef8bc528b237ab74f...,fwd,1.0,8.0
4,1499343600.001577000,192.168.10.25,216.58.219.226,49221,443,3561,806,00c1b114eb31002500a8c4600800,450000345e9f400040065d46c0a80a19d83adbe2,c04501bba9a50630d127ae518010ffcfc92d0000010108...,,bwd,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7187244,1499365265.817911000,206.126.112.142,192.168.10.16,443,42198,99781,4909563,0023ae9b8abf00c1b114eb310800,450005dcf0ae0000390681a8ce7e708ec0a80a10,01bba4d6f40edf7af598c863801000b7b3290000010108...,59210587b21ed3a8d3c3c35f16fa39c781507dfc743cfc...,bwd,0.0,0.0
7187245,1499365265.818102000,206.126.112.142,192.168.10.16,443,42198,99781,4911011,0023ae9b8abf00c1b114eb310800,450005dcf0af0000390681a7ce7e708ec0a80a10,01bba4d6f40ee522f598c863801000b7168a0000010108...,c92242f4b3a7d9a4c9cc0a701d995a7ac4c682158dadd2...,bwd,0.0,0.0
7187246,1499365265.818295000,206.126.112.142,192.168.10.16,443,42198,99781,4912459,0023ae9b8abf00c1b114eb310800,45000b84f0b0000039067bfece7e708ec0a80a10,01bba4d6f40eeacaf598c863801000b7153c0000010108...,988822bedfd4cf1c3958c978619bf4fa79fcdd10d45749...,bwd,0.0,0.0
7187247,1499365265.818487000,206.126.112.142,192.168.10.16,443,42198,99781,4915355,0023ae9b8abf00c1b114eb310800,45000b84f0b2000039067bfcce7e708ec0a80a10,01bba4d6f40ef61af598c863801000b7153c0000010108...,7c8dcb764403b28792b82bce3b6e9fb9a86771a3093069...,bwd,0.0,0.0


In [70]:
df_all.groupby('Direction')['Class Label'].value_counts()

Direction  Class Label
bwd        0.0            3892921
           11.0             29873
           8.0              10304
           9.0               3277
           10.0                59
fwd        0.0            3194751
           11.0             29881
           8.0              19755
           9.0               6361
           10.0                67
Name: Class Label, dtype: int64

In [71]:
df_all.to_pickle("Thursday_Packet_Level_Data.pkl")